In [45]:
%reset -f

In [115]:
import pandas as pd
import numpy as np
import random
import re
import gc

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import make_scorer, mean_absolute_error
from sklearn.feature_selection import SelectPercentile, f_regression
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.model_selection import cross_val_score, KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.linear_model import SGDClassifier
from sklearn.datasets import make_classification
import random
from sklearn.pipeline import make_pipeline
from sklearn.compose import TransformedTargetRegressor

In [47]:
gc.collect()

0

In [48]:
listings = pd.read_csv("train.csv")

In [49]:
listings.columns

Index(['Id', 'price', 'name', 'neighborhood_overview', 'host_id', 'host_since',
       'host_location', 'host_about', 'host_response_time',
       'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'host_listings_count', 'host_verifications', 'host_has_profile_pic',
       'host_identity_verified', 'neighbourhood_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'minimum_nights',
       'maximum_nights', 'minimum_minimum_nights', 'maximum_minimum_nights',
       'minimum_maximum_nights', 'maximum_maximum_nights',
       'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'has_availability',
       'availability_30', 'availability_60', 'availability_90',
       'availability_365', 'number_of_reviews', 'number_of_reviews_ltm',
       'number_of_reviews_l30d', 'first_review', 'last_review',
       'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cl

In [50]:
#random.sample(list(listings["neighborhood_overview"].values), 1)

In [51]:
positive_words = np.array(["close", "night life", "favorite", "famous",
                           "!", "amazing", "easy", "lovely", "easily",
                           "great", "shopping", "restaurants", "explore",
                           "historic", "hotspot", "relax",
                           "perfect", "unforgettable", "inspiring", "convenient",
                           "theaters", "attractions", "walking", "playground",
                           "park", "clubs", "quiet", "market",
                           "best", "library", "local", "sights",
                           "heart", "charming", "museum"])

In [52]:
def parse_descriptions(descriptions):
    parsed_info = {
        'bedrooms': [],
        'beds': [],
        'baths': []
    }

    for description in descriptions:
        brms = re.split(r'(\d+)\s*bedroom',description)
        bds = re.split(r'(\d+)\s*bed(?:s\b)?(?!room)',description)
        bths = re.split(r'(\d+(\.\d+)?)\s*(shared|private)?\s*bath',description)

        # print(brms)
        # print(bds)
        # print(bths)

        if len(brms) > 1:
            parsed_info["bedrooms"].append(float(brms[1]))
        else:
            parsed_info["bedrooms"].append(0)
        if len(bds) > 1:
            parsed_info["beds"].append(float(bds[1]))
        else: 
            parsed_info["beds"].append(0)
        if len(bths) > 1:
            parsed_info["baths"].append(float(bths[1]))
        else:
            parsed_info["baths"].append(0)

    return parsed_info

In [53]:
# descriptions = listings["name"].values
# apartment_stats  = parse_descriptions(descriptions)

In [54]:
# listings_cleaned = pd.concat([listings.drop(columns=["bedrooms"]), pd.DataFrame(apartment_stats)[["bedrooms", "baths"]]], axis=1)

In [55]:
# listings_cleaned["host_response_rate"] = listings_cleaned["host_response_rate"].apply(lambda r: float(r.split("%")[0]) if isinstance(r, str) else r)

In [56]:
# listings_cleaned.columns

In [57]:
# listings_cleaned["last_review"] = pd.to_datetime(listings_cleaned["last_review"])
# most_recent_date = listings_cleaned["last_review"].max()
# listings_cleaned["recency"] = 1+(most_recent_date-listings_cleaned["last_review"]).dt.days

In [58]:
def apt_type(description):
    return description.split(" ")[0].upper()

#listings_cleaned["apt_type"] = listings_cleaned["name"].apply(lambda d: apt_type(d))

In [59]:
# listings_cleaned["neighborhood_positivity"] = listings_cleaned["neighborhood_overview"].apply(
#     lambda d: np.sum([word in d.lower() for word in positive_words]) if not pd.isna(d) else 0
# )

In [60]:
# listings_cleaned["host_description_score"] = listings_cleaned["host_about"].apply(
#     lambda d: len(d) if not pd.isna(d) else 0
# )

# listings_cleaned["host_since"] = pd.to_datetime(listings_cleaned["host_since"])
# listings_cleaned["host_recency"] = 1+(most_recent_date-listings_cleaned["host_since"]).dt.days

In [61]:
# R = 6371

# x = listings_cleaned["latitude"].values
# y = listings_cleaned["longitude"].values

# def haversine(x0, y0, x_vector, y_vector):
#     x = np.deg2rad(x0)
#     y = np.deg2rad(y0)
#     lat = np.deg2rad(x_vector)
#     long = np.deg2rad(y_vector)
#     xdiff = x-lat
#     ydiff = y-long
    
#     d = np.sin(xdiff / 2)**2 + np.cos(x) * np.cos(lat) * np.sin(ydiff / 2)**2
#     c = 2 * np.arctan2(np.sqrt(d), np.sqrt(1-d))
#     km = R*c

#     return km

# def euclidean(x0, y0, x_vector, y_vector):
#     return np.sqrt(np.power((x0-x_vector),2)+np.power((y0+y_vector),2))

# def compute_weights(epsilon, d, p=2):
#     return np.power(epsilon-d, p)/np.power(epsilon, p)

#compute_weights(3, np.array([random.uniform(0, 3) for _ in range(10)]))

In [62]:
# #Spatial dependency
# neighborhood_prices = []

# epsilon = 1

# for i, data in listings_cleaned.iterrows():

#     x0 = data["latitude"]
#     y0 = data["longitude"]
    
#     d = haversine(x0, y0, x, y)

#     indices = np.where(d <= epsilon)[0]
#     neighborhood = indices[indices != i]

#     i_type = listings_cleaned.iloc[i]["property_type"]
#     same_types = np.where(listings_cleaned["property_type"].values == i_type)[0]

#     neighborhood = np.intersect1d(same_types, neighborhood)

#     weights = compute_weights(epsilon, d[neighborhood])
#     if len(weights) != 0:
#         neighborhood_price = sum(listings_cleaned.iloc[neighborhood]["price"]*weights)/sum(weights)
#     else:
#         neighborhood_price = None
#     #print(neighborhood_price)
#     #neighborhood_price = np.mean(listings_cleaned.iloc[neighborhood]["price"])

#     prices = neighborhood_prices.append(neighborhood_price)

# listings_cleaned["neighborhood_price"] = neighborhood_prices
# neighborhood_prices


In [63]:
# listings_cleaned.columns

In [64]:
# dummies = ["host_location", "host_identity_verified", "property_type",
#                   "room_type", "has_availability"]
# continuous = [    "accommodates", "baths", "bedrooms", "beds", "recency", "neighborhood_price"]

In [65]:
# listings_cleaned[["price"]+continuous].corr()

In [66]:
# pipe_numeric = Pipeline(steps=[
#   ('impute',KNNImputer()),
#   ('standardize', StandardScaler())
# ])

# pipe_categorical = Pipeline(steps=[
#   ('impute',SimpleImputer(strategy='most_frequent')),
#   ('onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
# ])

# preprocessor = ColumnTransformer(
#     transformers=[
#         ("numerical", pipe_numeric, continuous),
#         ("categorical", pipe_categorical, dummies)
#     ]
# )

# knn_pipe = Pipeline(steps=[
#     ("preprocessors", preprocessor),
#     ('poly_features', PolynomialFeatures(degree=3, include_bias=False)),
#     ('model', KNeighborsRegressor(n_neighbors=10, weights='distance'))
# ])

In [67]:
# params = {
#   #'preprocessors__categorical__select_percentile__percentile': [50],
#   'preprocessors__numerical__impute__n_neighbors': list(range(5, 101, 5)),
#   'poly_features__degree': [3],
#   'model__n_neighbors': list(range(5, 101, 5)),
#   'model__weights': ['distance']
# }

# X = listings_cleaned[continuous+dummies]
# Y = listings_cleaned["price"]
# mae = make_scorer(mean_absolute_error, greater_is_better=False)
# X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.5, random_state=486)
# # knn_pipe.fit(X_train, Y_train)

In [68]:
# rs = RandomizedSearchCV(knn_pipe, param_distributions=params, scoring=mae, cv=5, random_state=486)

In [69]:
#We don't have enough memory to run this
#rs.fit(X,Y)

In [70]:
#rs.best_params_

In [71]:
# #KNN Model 1
# cv = KFold(n_splits=5,
#            shuffle=True,
#            random_state=0)

# pipe_numeric = Pipeline(steps=[
#   ('impute',KNNImputer(n_neighbors=10)),
#   ('standardize', StandardScaler())   
# ])

# knn_pipe = Pipeline(steps=[
#     ("preprocessors", preprocessor),
#     ('poly_features', PolynomialFeatures(degree=3, include_bias=False)),
#     ('model', KNeighborsRegressor(n_neighbors=10, weights='distance'))
# ])

# # knn_pipe.fit(X_train, Y_train)
# # knn_yhat = knn_pipe.predict(X_test)

# #knn_cv_error = cross_val_score(knn_pipe, X, Y, cv=cv, scoring=mae)
# #-np.mean(knn_cv_error)

In [72]:
# #KNN Model 2
# cv = KFold(n_splits=5,
#            shuffle=True,
#            random_state=0)

# pipe_numeric = Pipeline(steps=[
#   ('impute',KNNImputer(n_neighbors=10)),
#   ('standardize', StandardScaler())   
# ])

# knn_pipe = Pipeline(steps=[
#     ("preprocessors", preprocessor),
#     ('poly_features', PolynomialFeatures(degree=3, include_bias=False)),
#     ('model', KNeighborsRegressor(n_neighbors=20, weights='distance'))
# ])

# # knn_pipe.fit(X_train, Y_train)
# # knn_yhat = knn_pipe.predict(X_test)

# #knn_cv_error = cross_val_score(knn_pipe, X, Y, cv=cv, scoring=mae)
# #-np.mean(knn_cv_error)

In [73]:
# #KNN Model 3
# cv = KFold(n_splits=10,
#            shuffle=True,
#            random_state=0)

# pipe_numeric = Pipeline(steps=[
#   ('impute',KNNImputer(n_neighbors=15)),
#   ('standardize', StandardScaler())   
# ])

# knn_pipe = Pipeline(steps=[
#     ("preprocessors", preprocessor),
#     ('poly_features', PolynomialFeatures(degree=1, include_bias=False)),
#     ('model', KNeighborsRegressor(n_neighbors=15, weights='distance'))
# ])

# # knn_pipe.fit(X_train, Y_train)
# # knn_yhat = knn_pipe.predict(X_test)

# #knn_cv_error = cross_val_score(knn_pipe, X, Y, cv=cv, scoring=mae)
# #-np.mean(knn_cv_error)

In [74]:
#Ridge regression
# alphas = np.logspace(-6, 6, 30)

# ridge_pipe = Pipeline(steps=[
#     ("preprocessors", preprocessor),
#     ('poly_features', PolynomialFeatures(degree=1, include_bias=False)),
#     ('model', RidgeCV(alphas=np.logspace(-6, 6, 30), cv=10))
# ])

# ridge_pipe.fit(X_train, Y_train)
# yhat = ridge_pipe.predict(X_test)
# mean_absolute_error(Y_test, yhat)


In [75]:
#Lasso regression
# lasso_pipe = Pipeline(steps=[
#     ("preprocessors", preprocessor),
#     ('poly_features', PolynomialFeatures(degree=1, include_bias=False)),
#     ('model', LassoCV(alphas=np.logspace(-6, 6, 30), cv=10))
# ])

# lasso_pipe.fit(X_train, Y_train)
# lasso_yhat = ridge_pipe.predict(X_test)
# mean_absolute_error(Y_test, lasso_yhat)

In [90]:
dummies = ["host_location", "host_identity_verified", "property_type",
                  "room_type", "has_availability"]
continuous = [    "accommodates", "baths", "bedrooms", "beds", "recency", "latitude", "longitude"]#, "neighborhood_price"]
#         'review_scores_rating', 'review_scores_accuracy',
#         'review_scores_cleanliness', 'review_scores_checkin',
#        'review_scores_communication', 'review_scores_location',
#        'review_scores_value',
#        "neighborhood_positivity"]

In [95]:
#Final model
pipe_numeric = Pipeline(steps=[
  ('impute',KNNImputer(n_neighbors=10)),
  ('standardize', StandardScaler())   
])

pipe_categorical = Pipeline(steps=[
  ('impute',SimpleImputer(strategy='most_frequent')),
  ('onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("numerical", pipe_numeric, continuous),
        ("categorical", pipe_categorical, dummies)
    ]
)

knn_pipe = Pipeline(steps=[
    ("preprocessors", preprocessor),
    ('poly_features', PolynomialFeatures(degree=1, include_bias=False)),
    ('model', KNeighborsRegressor(n_neighbors=10, weights='distance'))
])


test = pd.read_csv("test.csv")

In [92]:
def compute_weights(epsilon, d, p=2):
    return np.power(epsilon-d, p)/np.power(epsilon, p)

R = 6371

def haversine(x0, y0, x_vector, y_vector):
    x = np.deg2rad(x0)
    y = np.deg2rad(y0)
    lat = np.deg2rad(x_vector)
    long = np.deg2rad(y_vector)
    xdiff = x-lat
    ydiff = y-long
    
    d = np.sin(xdiff / 2)**2 + np.cos(x) * np.cos(lat) * np.sin(ydiff / 2)**2
    c = 2 * np.arctan2(np.sqrt(d), np.sqrt(1-d))
    km = R*c

    return km

def create_features(df, base=listings):

    cleaned = pd.concat([df.drop(columns=["bedrooms"]), pd.DataFrame(parse_descriptions(df["name"].values))[["bedrooms", "baths"]]], axis=1)
    cleaned["host_response_rate"] = cleaned["host_response_rate"].apply(lambda r: float(r.split("%")[0]) if isinstance(r, str) else r)
    cleaned["last_review"] = pd.to_datetime(cleaned["last_review"])
    most_recent_date = cleaned["last_review"].max()
    cleaned["recency"] = 1+(most_recent_date-cleaned["last_review"]).dt.days
    cleaned["apt_type"] = cleaned["name"].apply(lambda d: apt_type(d))
    cleaned["neighborhood_positivity"] = cleaned["neighborhood_overview"].apply(
        lambda d: np.sum([word in d.lower() for word in positive_words]) if not pd.isna(d) else 0
    )
    cleaned["host_description_score"] = cleaned["host_about"].apply(
        lambda d: len(d) if not pd.isna(d) else 0
    )

    cleaned["host_since"] = pd.to_datetime(cleaned["host_since"])
    cleaned["host_recency"] = 1+(most_recent_date-cleaned["host_since"]).dt.days

    #Spatial dependency
    neighborhood_prices = []

    epsilon = 1

    for i, data in cleaned.iterrows():

        x0 = data["latitude"]
        y0 = data["longitude"]
        
        d = haversine(x0, y0, base["latitude"].values, base["longitude"].values)

        indices = np.where(d <= epsilon)[0]
        neighborhood = indices[indices != i]

        i_type = cleaned.iloc[i]["property_type"]
        same_types = np.where(base["property_type"].values == i_type)[0]

        neighborhood = np.intersect1d(same_types, neighborhood)

        weights = compute_weights(epsilon, d[neighborhood])
        if len(weights) != 0:
            neighborhood_price = sum(base.iloc[neighborhood]["price"]*weights)/sum(weights)
        else:
            neighborhood_price = None

        prices = neighborhood_prices.append(neighborhood_price)

    cleaned["neighborhood_price"] = neighborhood_prices

    return cleaned

listings_cleaned = create_features(listings).copy()

X = listings_cleaned[continuous+dummies]
Y = listings_cleaned["price"]

test_cleaned = create_features(test).copy()

In [105]:
shuffle_indices = np.random.permutation(len(X))

X_shuffled = X.iloc[shuffle_indices].reset_index(drop=True)
Y_shuffled = Y.iloc[shuffle_indices].reset_index(drop=True)

batches = 10
batch_size = int(np.floor(len(X_shuffled)/batches))

predictions = []

for i in range(0, len(X_shuffled), batch_size):
    
    if i+2*batch_size > len(X_shuffled):
        batch_size = len(X_shuffled)-i

        X_batch = X_shuffled.iloc[i:i+batch_size]
        Y_batch = Y_shuffled.iloc[i:i+batch_size]
        
        knn_pipe.fit(X_batch, Y_batch)
        predictions.append(knn_pipe.predict(test_cleaned[continuous+dummies]))

        print("100%")

        break;

    else:

        X_batch = X_shuffled.iloc[i:i+batch_size]
        Y_batch = Y_shuffled.iloc[i:i+batch_size]
        
        knn_pipe.fit(X_batch, Y_batch)
        predictions.append(knn_pipe.predict(test_cleaned[continuous+dummies]))

        print((i+batch_size)/len(X_shuffled))

0.09997381170616734
0.19994762341233469
0.29992143511850206
0.39989524682466937
0.49986905853083674
0.5998428702370041
0.6998166819431714
0.7997904936493387
0.8997643053555061
100%


In [106]:
new_predictions = np.mean(np.array(predictions), axis=0)

In [108]:
predictions = pd.DataFrame(test_cleaned["Id"].copy())
predictions["price"] = new_predictions

In [109]:
predictions

,Id,price
0,PSJEN,83.457333
1,PVZW7,190.418903
2,EJLAM,72.801524
3,SDHPB,65.952457
4,MJGYX,388.607606
...,...,...
6542,2TAJ9,534.662046
6543,YFHBN,87.178555
6544,Z0TCA,68.795212
6545,VBCBJ,272.957071


In [110]:
predictions.to_csv("KNNPredictions.csv", index=False)

In [142]:

dummies = ["host_location", "host_identity_verified", "property_type",
                  "room_type", "has_availability"]
continuous = [    "accommodates", "baths", "bedrooms", "beds", "recency", "neighborhood_price"]

X = listings_cleaned[continuous+dummies]

pipe_numeric = Pipeline(steps=[
  ('impute',KNNImputer(n_neighbors=10)),
  ('poly_features', PolynomialFeatures(degree=3, include_bias=False))   
])

pipe_categorical = Pipeline(steps=[
  ('impute',SimpleImputer(strategy='most_frequent')),
  ('onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("numerical", pipe_numeric, continuous),
        ("categorical", pipe_categorical, dummies)
    ]
)

preprocessor.fit(X)

ColumnTransformer(transformers=[('numerical',
                                 Pipeline(steps=[('impute',
                                                  KNNImputer(n_neighbors=10)),
                                                 ('poly_features',
                                                  PolynomialFeatures(degree=3,
                                                                     include_bias=False))]),
                                 ['accommodates', 'baths', 'bedrooms', 'beds',
                                  'recency', 'neighborhood_price']),
                                ('categorical',
                                 Pipeline(steps=[('impute',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False))]),
                                 ['host_location', 'host_identity_verified',
                                  'property_type', 'room_type',
                                  'has_availability'])])

In [138]:
X_transformed = preprocessor.transform(X)

rf = RandomForestRegressor(n_estimators=100, warm_start=True)

shuffle_indices = np.random.permutation(len(X))

X_shuffled = X_transformed[shuffle_indices]
Y_shuffled = Y.iloc[shuffle_indices].reset_index(drop=True)

batch_size = 1000

for i in range(0, len(X_shuffled), batch_size):
    
    X_batch = X_shuffled[i:i+batch_size]
    Y_batch = Y_shuffled.iloc[i:i+batch_size]
    
    rf.n_estimators += 100
    rf.fit(X_batch, Y_batch)

    print((i+batch_size)/len(X_shuffled))

0.065470734581642
0.130941469163284
0.19641220374492602
0.261882938326568
0.32735367290821005
0.39282440748985203
0.458295142071494
0.523765876653136
0.5892366112347781
0.6547073458164201
0.7201780803980621
0.7856488149797041
0.8511195495613461
0.916590284142988
0.9820610187246301
1.047531753306272


In [139]:
#MAE of our data set
rf_yhat = rf.predict(X_transformed)
mean_absolute_error(Y, rf_yhat)

190.26049480085342

In [140]:
predictions = pd.DataFrame(test_cleaned["Id"].copy())
predictions["price"] = rf.predict(preprocessor.transform(test_cleaned[continuous+dummies]))
predictions

,Id,price
0,PSJEN,77.898824
1,PVZW7,91.969412
2,EJLAM,66.628235
3,SDHPB,55.444118
4,MJGYX,332.873529
...,...,...
6542,2TAJ9,834.945294
6543,YFHBN,69.057647
6544,Z0TCA,83.414706
6545,VBCBJ,276.485882


In [141]:
predictions.to_csv("rf_predictions.csv", index=False)

##### 

#### 